In [1]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter,Language

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [ ]:
repo_path = "/Users/yogeshagrawal/Desktop/Gen AI/27.Code_Explainer/code_repo"
# Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", to_path = repo_path)

In [5]:
loader = GenericLoader.from_filesystem(
    path=repo_path,
    glob="**/*.py",
    parser = LanguageParser(language=Language.PYTHON),
    show_progress=True,
)

documents = loader.load()

/Users/yogeshagrawal/Desktop/Gen AI/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 7/7 [00:00<00:00, 920.21it/s]


In [6]:
RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 500,
    chunk_overlap = 50
)
text_chunks = RecursiveCharacterTextSplitter.split_documents(documents)

In [8]:
vectorstore = FAISS.from_documents(documents= text_chunks, embedding=OpenAIEmbeddings())
# Save to local folder
vectorstore.save_local("/Users/yogeshagrawal/Desktop/Gen AI/27.Code_Explainer/faiss_index")

# faiss_store = FAISS.load_local("faiss_index", embeddings=OpenAIEmbeddings())

In [11]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={"k": 5})

In [9]:
llm = ChatOpenAI()

In [10]:
memory = ConversationSummaryMemory(llm = llm, memory_key="chat_history", return_messages=True)

/var/folders/7l/dw4qxb8d2qb48m997dzvxtsc0000gn/T/ipykernel_74735/2314359002.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm = llm, memory_key="chat_history", return_messages=True)


In [15]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, verbose=True)

In [13]:
question = "what is download_hugging_face_embeddings function ?"

In [16]:
result = qa(question)
print(result['answer'])

/var/folders/7l/dw4qxb8d2qb48m997dzvxtsc0000gn/T/ipykernel_74735/79176006.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions
    return embeddings

from src.helper import load_pdf_file, text_split, download_hugging_face_embeddings
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os


load_dotenv()

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
